Jamison Hunter

June 29, 2023

# IMDB Data Analysis & Recommendations

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

In [2]:
# defining urls as variables
# Source: https://datasets.imdbws.com/
basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"
akas_url="https://datasets.imdbws.com/title.akas.tsv.gz"
ratings_url="https://datasets.imdbws.com/title.ratings.tsv.gz"

In [3]:
dfb = pd.read_csv(basics_url, sep='\t', low_memory=False)

In [4]:
dfa = pd.read_csv(akas_url, sep='\t', low_memory=False)

In [5]:
dfr = pd.read_csv(ratings_url, sep='\t', low_memory=False)

In [6]:
dfb.info()
dfb.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9978989 entries, 0 to 9978988
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 685.2+ MB


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [7]:
dfa.info()
dfa.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36432853 entries, 0 to 36432852
Data columns (total 8 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   titleId          object
 1   ordering         int64 
 2   title            object
 3   region           object
 4   language         object
 5   types            object
 6   attributes       object
 7   isOriginalTitle  object
dtypes: int64(1), object(7)
memory usage: 2.2+ GB


,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [8]:
dfr.info()
dfr.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1326238 entries, 0 to 1326237
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1326238 non-null  object 
 1   averageRating  1326238 non-null  float64
 2   numVotes       1326238 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 30.4+ MB


,tconst,averageRating,numVotes
0,tt0000001,5.7,1983
1,tt0000002,5.8,265
2,tt0000003,6.5,1840
3,tt0000004,5.5,178
4,tt0000005,6.2,2623


In [9]:
# replacing \N values with np.nan for all data frames loaded so far
dfb = dfb.replace({'\\N':np.nan})
dfa = dfa.replace({'\\N':np.nan})
dfr = dfr.replace({'\\N':np.nan})

In [10]:
# Eliminate movies that are null for runtimeMinute, genres, and startYear
dfb = dfb.dropna(subset=['runtimeMinutes','genres','startYear'])
dfb['startYear'] = dfb['startYear'].astype(float)
# keeping startYear 2000-2022
dfb = dfb[(dfb['startYear']>=2000)&(dfb['startYear']<2022)]
# allowing only Movies in the data frame
dfb.drop(dfb.loc[dfb['titleType']!="movie"].index, inplace=True)

In [11]:
# Exclude movies that are included in the documentary category.
is_documentary = dfb['genres'].str.contains('documentary',case=False)
dfb = dfb[~is_documentary]

In [12]:
# Filter the basics table down to only include the US by using the filter akas dataframe
df_us = dfa.drop(columns = ["ordering", "title", "language", "types", "attributes", "isOriginalTitle"])
df_us = df_us[~(df_us["region"] != "US")]
keepers = dfb['tconst'].isin(df_us['titleId'])
dfb = dfb[keepers]
# stating the number of rows and columns in the filtered data set
print(f"This data set has {dfb.shape[0]} rows and {dfb.shape[1]} columns.")
dfb.head()

This data set has 81835 rows and 9 columns.


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
42384,tt0043139,movie,Life of a Beijing Policeman,Wo zhe yi bei zi,0,2013.0,NaN,120,"Drama,History"
61115,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
67667,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
86799,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"


Next I will filter the akas data frame in order to only include US movies.

In [13]:
# filtering akas frame to only include US films
dfa = dfa[~(dfa["region"] != "US")]
dfa["region"].unique()

array(['US'], dtype=object)

I will do the same with ratings so that only US movies have ratings available.

In [14]:
df_us = dfa.drop(columns = ["ordering", "title", "language", "types", "attributes", "isOriginalTitle"])
df_us = df_us[~(df_us["region"] != "US")]
keepers = dfr['tconst'].isin(df_us['titleId'])
dfr = dfr[keepers]
# stating the number of rows and columns in the filtered data set
print(f"This data set has {dfr.shape[0]} rows and {dfr.shape[1]} columns.")
dfr.head()

This data set has 502645 rows and 3 columns.


,tconst,averageRating,numVotes
0,tt0000001,5.7,1983
1,tt0000002,5.8,265
4,tt0000005,6.2,2623
5,tt0000006,5.1,182
6,tt0000007,5.4,821


In [15]:
# example making new folder with os
os.makedirs('Data/',exist_ok=True) 
# Confirm folder created
os.listdir("Data/")

['.ipynb_checkpoints']

In [16]:
dfb.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
42384,tt0043139,movie,Life of a Beijing Policeman,Wo zhe yi bei zi,0,2013.0,NaN,120,"Drama,History"
61115,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
67667,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
86799,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"


In [17]:
## Save current dataframe to file.
dfb.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)

In [18]:
dfa.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0


In [19]:
## Save current dataframe to file.
dfa.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)

In [20]:
dfr.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1983
1,tt0000002,5.8,265
4,tt0000005,6.2,2623
5,tt0000006,5.1,182
6,tt0000007,5.4,821


In [21]:
## Save current dataframe to file.
dfr.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)

I will confirm that the data frames saved properly and can be accessed. 

In [22]:
# Open saved file and preview again
dfb = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
dfb.info()
dfb.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81835 entries, 0 to 81834
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          81835 non-null  object 
 1   titleType       81835 non-null  object 
 2   primaryTitle    81835 non-null  object 
 3   originalTitle   81835 non-null  object 
 4   isAdult         81835 non-null  int64  
 5   startYear       81835 non-null  float64
 6   endYear         0 non-null      float64
 7   runtimeMinutes  81835 non-null  int64  
 8   genres          81835 non-null  object 
dtypes: float64(2), int64(2), object(5)
memory usage: 5.6+ MB


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0043139,movie,Life of a Beijing Policeman,Wo zhe yi bei zi,0,2013.0,NaN,120,"Drama,History"
2,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
3,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"


In [23]:
# Open saved file and preview again
dfa = pd.read_csv("Data/title_akas.csv.gz", low_memory = False)
dfa.info()
dfa.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1448892 entries, 0 to 1448891
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   titleId          1448892 non-null  object 
 1   ordering         1448892 non-null  int64  
 2   title            1448892 non-null  object 
 3   region           1448892 non-null  object 
 4   language         3978 non-null     object 
 5   types            980852 non-null   object 
 6   attributes       46896 non-null    object 
 7   isOriginalTitle  1447550 non-null  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 88.4+ MB


,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0


In [24]:
# Open saved file and preview again
dfr = pd.read_csv("Data/title_ratings.csv.gz", low_memory = False)
dfr.info()
dfr.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502645 entries, 0 to 502644
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         502645 non-null  object 
 1   averageRating  502645 non-null  float64
 2   numVotes       502645 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 11.5+ MB


,tconst,averageRating,numVotes
0,tt0000001,5.7,1983
1,tt0000002,5.8,265
2,tt0000005,6.2,2623
3,tt0000006,5.1,182
4,tt0000007,5.4,821
